<a href="https://colab.research.google.com/github/MGuvenc/Novelty/blob/main/train_novelty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import joblib, warnings, datetime, logging, cx_Oracle

from sklearn.ensemble import IsolationForest
from sklearn.exceptions import ConvergenceWarning

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [18]:
def write_date():
    return datetime.datetime.strftime(datetime.datetime.now(), '%Y/%m/%d %H:%M  -  ')

def yazdir(x):
    try:
        print(x)
        f.write('\n'+str(x))
        logging.info(x)
    except Exception as e:
        yazdir('ERR#2 : Logging execution error' + str(e))

f = ''
try:
    f = open('ow_engine.log', 'w')
    logging.basicConfig(filename='OWengine.log' )
except Exception as e:
    yazdir('ERR#1 :  Logging initiation error' + str(e))

#yazdir(datetime.datetime.strftime(datetime.datetime.now(), '%Y%m%d%H%M_log.txt'),"w+")
yazdir(write_date() + ' Vadeli TL ürünü için çalışma başlatıldı.\n')

yazdir('konfigürasyon değişkenleri yüklendi')

2023/09/26 06:56  -   Vadeli TL ürünü için çalışma başlatıldı.

konfigürasyon değişkenleri yüklendi


In [ ]:
sql = '''
     SELECT * FROM xxxx.xxxxxx_temizlenmis
     '''

dsn_tns = cx_Oracle.makedsn('xx.xx.xxx.xx', 'xxxx', service_name='xxxxxxx')
conn = cx_Oracle.connect(user=r'xxxxxxxxxx', password='xxxxxxx', dsn=dsn_tns)
c = conn.cursor()

In [ ]:
try:
    data = pd.read_sql_query(sql, conn)
    orig_data = data.copy()

    conn.close()
    yazdir(' Veri Setleri Okundu.')
except Exception as e:
    yazdir('ERR#3 : Dosya açılma sırasında hata oldu.'+ str(e))

In [3]:
data['TARIH'] = data[('MUHASEBE_TR')].values.astype('float64')
data.rename(columns={'BAKIYE': 'GIDER'}, inplace=True)
id = data.groupby('KUME')

In [ ]:
def train(id):
    aylik_data = pd.DataFrame()
    models = {}
    for unique_id, group_data in id:
        model = IsolationForest(contamination=0.1)
        model.fit(group_data[['tarih', 'gider']])
        models[unique_id] = model
    return models

In [ ]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = executor.map(train, id)

In [5]:
joblib.dump(results, 'models.pkt')

['models.pkt']